## Motivation

The scikit package for python will be one of the most important ones during the master programs. Therefore I came up with the idea to get a overview what topics are coverd on the documentation webpage and especially which external resources are provided. The apporoch would be to start at the highest level of the webpage and then go down the tree shaped website directory. I limit the analysis to 2 levels below the starting point, otherwise it would be computational challenging. 

### Packages

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import queue
from concurrent.futures import ThreadPoolExecutor

### Define classes and functions needed

In [2]:
def get_class(DRs):
    try:
        if len(DRs.get("class")) > 1: 
            class_lap = str("")
            for i in range(len(DRs.get("class"))):
                class_lap = str(class_lap + DRs.get("class")[i])
            return class_lap
        else: 
            return DRs.get("class")
    except:
        "Not Labeled"

In [3]:
def website_type(webdir):
    if (webdir[0:4] == "http") or (webdir[-3:] == "pdf"):
        return "foreign"
    else: 
        return "scikit"

In [4]:
def get_links_scikit(webdir):
    if website_type(webdir) == "scikit":
        URL = "http://scikit-learn.org/stable/"
        # webdir = "documentation.html"
        next_url = str(URL + webdir)
        soup = BeautifulSoup(requests.get(next_url).content, "lxml")
        link_data = pd.DataFrame([])
        if len(soup.select("div.body a")) > 0:
            for DRs in soup.select("div.body a"):
                link_data = link_data.append(pd.DataFrame({"Website": URL, "Location": webdir, 
                                                           "Links": DRs.get("href"), "Type": get_class(DRs)}, 
                                                          index=[0])
                                             , ignore_index=True)
        else: 
            link_data = pd.DataFrame({"Website": URL, "Location": webdir, "Links": "No Link here", "Type": "Not Labeled"}, 
                                                      index=[0])
    else: 
        link_data = pd.DataFrame({"Website": webdir, "Location": webdir, "Links": "No Link here", "Type": "Not Labeled"}, 
                                                      index=[0])
    return link_data

In [5]:
def simple_worker(i):
    while True:
        item = q.get()
        if item == 'break':
            break
        results = get_links_scikit(item)
        q.task_done()
        if (q.unfinished_tasks in [2000,1500,1000,500,10]) or (q.unfinished_tasks < 10):
            print("Task_done & amount of unfinished sub tasks: " + str(q.unfinished_tasks))
        return results

In [6]:
def hard_worker(i):
    while True:
        print("Amount of unfinished main tasks: " + str(m.unfinished_tasks))
        item = m.get()
        if item == 'break':
            break
        results = get_links_scikit(item)
        if any(results):
            for x in results["Links"]:
                q.put(x)
        m.task_done()
        print("Task_done & amount of unfinished tasks: " + str(q.unfinished_tasks))
        return results

### Code 

In [ ]:
# Simple worker with given list of links
initial_tasks = list(get_links_scikit("documentation.html")["Links"])
#del initial_tasks[3:5]
q = queue.Queue()
for y in initial_tasks: 
    q.put(y)
with ThreadPoolExecutor(max_workers=500) as executor:
    futures = executor.map(simple_worker,range(len(initial_tasks)))
    q.join()
    for i in range(500):
        q.put('break')

links = pd.DataFrame([])
for value in futures:
    links = links.append(value,ignore_index=True)

In [7]:
# Hard worker
m = queue.Queue()
q = queue.Queue()

initial_tasks = list(get_links_scikit("documentation.html")["Links"])
for y in initial_tasks: 
    m.put(y)
with ThreadPoolExecutor(max_workers=m.unfinished_tasks) as executor:
    executor.map(hard_worker,range(m.unfinished_tasks))
    m.join()
    for i in range(m.unfinished_tasks):
        m.put('break')

Amount of unfinished main tasks: 13
Amount of unfinished main tasks: 13
Amount of unfinished main tasks: 13
Amount of unfinished main tasks: 13
Task_done & amount of unfinished tasks: 1
Amount of unfinished main tasks: 12
Task_done & amount of unfinished tasks: 2
Amount of unfinished main tasks: 11
Amount of unfinished main tasks: 11
Amount of unfinished main tasks: 11
Amount of unfinished main tasks: 11
Amount of unfinished main tasks: 11Amount of unfinished main tasks: 11
Amount of unfinished main tasks: 11

Amount of unfinished main tasks: 11
Task_done & amount of unfinished tasks: 33
Task_done & amount of unfinished tasks: 61
Task_done & amount of unfinished tasks: 62
Task_done & amount of unfinished tasks: 109
Task_done & amount of unfinished tasks: 185
Task_done & amount of unfinished tasks: 273
Task_done & amount of unfinished tasks: 323
Task_done & amount of unfinished tasks: 364
Task_done & amount of unfinished tasks: 928
Task_done & amount of unfinished tasks: 1407
Task_done 

In [ ]:
with ThreadPoolExecutor(max_workers= (q.unfinished_tasks+8)) as executor:
    futures = executor.map(simple_worker,range((q.unfinished_tasks+8)))
    q.join()
    for i in range((q.unfinished_tasks+8)):
        q.put('break')

Task_done & amount of unfinished sub tasks: 2000
Task_done & amount of unfinished sub tasks: 1500
Task_done & amount of unfinished sub tasks: 1000
Task_done & amount of unfinished sub tasks: 500


In [10]:
q.unfinished_tasks


2080

In [ ]:
links = pd.DataFrame([])
for value in futures:
    links = links.append(value,ignore_index=True)
# links

### Saving data

In [ ]:
links.to_csv("scikit_links.csv")

### Explore external links

In [ ]:
to_explore = pd.DataFrame(links[links.Type == "referenceexternal"])
res = []
full = []
res2 = []
for values in to_explore["Links"]:
    res2.append([s for s in values.split("/")])
    res.append([s for s in values.split(".") if "/" not in s])
    full.append(values.split("."))
res
full
res2